In [1]:
%matplotlib inline
import glob
import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras import backend as K
from keras import metrics
import mido
import numpy
from seya.layers.variational import *
import skimage.io

Using TensorFlow backend.


In [ ]:
notes = 36
lookback = 32

In [ ]:
# input_vector = Input((None, notes))
# input_layer = TimeDistributed(Dense(notes, activation='sigmoid'))(input_vector)
# previous_layer = input_layer
# for units in reversed(range(notes // 6, notes + 1, 4)):
#     hidden_layer = Bidirectional(LSTM(units, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(previous_layer)
#     previous_layer = hidden_layer
# middle_layer = TimeDistributed(Dense(notes // 6, activation='sigmoid'))(previous_layer)
# previous_layer = middle_layer
# for units in range(notes // 6, notes + 1, 4):
#     hidden_layer = Bidirectional(LSTM(units, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(previous_layer)
#     previous_layer = hidden_layer
# output_layer = TimeDistributed(Dense(notes, activation='sigmoid'))(previous_layer)
# model = Model(input_vector, output_layer)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
# input_size = 1 + notes + 8 + 16
# input_layer = Input((None, input_size))
# previous_layer = input_layer
# for units in reversed(range(input_size // 6, input_size + 1, 8)):
#     hidden_layer = Bidirectional(LSTM(units, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(previous_layer)
#     previous_layer = hidden_layer
# middle_layer = TimeDistributed(Activation('sigmoid'))(previous_layer)
# previous_layer = middle_layer
# for units in range(input_size // 6, input_size + 1, 8):
#     hidden_layer = Bidirectional(LSTM(units, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(previous_layer)
#     previous_layer = hidden_layer
# output_layer = TimeDistributed(Dense(input_size, activation='sigmoid'))(previous_layer)
# model = Model(input_layer, output_layer)
# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])

In [ ]:
# model.summary()

In [ ]:
# x = numpy.zeros((1, 16, 57), dtype=int)
# for i in range(16):
#     x[:, i, i:i+4] = 1

In [ ]:
# x

In [ ]:
# y = numpy.invert(numpy.roll(x, -1, axis=1)) + 2

In [ ]:
# y

In [ ]:
# model.fit(x, y, epochs=10000)

In [ ]:
# numpy.round(model.predict(x)).astype(int)

In [ ]:
# midi = mido.MidiFile('/home/santiago/Projects/MusicGenerator/data/midi/6.mid')

In [ ]:
# test = []
# for track in midi.tracks:
#     if track.name == 'Melody':
#         for message in track:
#             test.append(message)
# len(test)

In [ ]:
def chords_from_midi(midi_file):
    data = []
    midi = mido.MidiFile(midi_file)
    for track in midi.tracks:
        if track.name == 'Chords':
            for message in track:
                if message.type in ['note_on', 'note_off']:
                    data.append((1 if message.type == 'note_on' else 0, message.note, message.velocity, message.time))
    assert data
    return numpy.array(data)


def encode_chords(chords):
    data = []
    minimum = min(chords[:, 1])
    prev_type = None
    prev_veloc = None
    encoded = None
    for chord in chords:
        if prev_type != chord[0] or prev_veloc != chord[2] or chord[3] != 0:
            if encoded is not None:
                data.append(encoded)
            encoded = numpy.zeros((1 + notes + 8 + 16,))
            encoded[0] = chord[0]
            encoded[1 + notes:1 + notes + 8] = [int(x) for x in format(chord[2], '08b')]
            encoded[1 + notes + 8:1 + notes + 8 + 16] = [int(x) for x in format(chord[3], '016b')]
        encoded[1 + chord[1] - minimum] = 1
        prev_type = chord[0]
        prev_veloc = chord[2]
    return numpy.array(data)


def load_chords(midi_dir):
    progressions = []
    midi_files = sorted(glob.glob(os.path.join(midi_dir, '*.mid')) + glob.glob(os.path.join(midi_dir, '*.midi')))
    for midi_file in midi_files:
        try:
            chords = chords_from_midi(midi_file)
            encoded = encode_chords(chords)
            prepared = input_output(encoded)
            progressions.append(prepared)
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            print("Skipping", midi_file)
    return progressions

def input_output(sequence):
    x = []
    y = []
    for i in range(len(sequence)):
        if i == 0:
            x.append(numpy.zeros((1, 1, 1 + notes + 8 + 16)).astype(int))
        elif i < lookback:
            x.append(numpy.vstack([numpy.zeros((1, 1 + notes + 8 + 16)), sequence[:i, :]])[numpy.newaxis, :, :].astype(int))
        else:
            x.append(numpy.vstack([numpy.zeros((1, 1 + notes + 8 + 16)), sequence[i - lookback:i, :]])[numpy.newaxis, :, :].astype(int))
        y.append(sequence[numpy.newaxis, i, :].astype(int))
    return (x, y)

def generator(data):
    while True:
        for sequence in data:
            for i in range(len(sequence[0])):
                yield (sequence[0][i], sequence[1][i])

In [ ]:
# data = chords_from_midi('/home/santiago/Projects/MusicGenerator/data/midi/6.mid')

In [ ]:
# data

In [ ]:
# X = encode_chords(data)

In [ ]:
# for a in x:
#     print(a)

In [ ]:
# train_data = input_output(X)

In [ ]:
# train_data[0][4]

In [ ]:
# input_size = 1 + notes + 8 + 16
# input_layer = Input((None, input_size))
# x = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(input_layer)
# x = Conv1D(128, 3, padding='causal')(x)
# x = Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(x)
# x = Conv1D(64, 3, padding='causal')(x)
# x = Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(x)
# x = Conv1D(32, 3, padding='causal')(x)
# x = Bidirectional(LSTM(16, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(x)
# x = Conv1D(16, 3, padding='causal')(x)
# middle_layer = TimeDistributed(Activation('sigmoid'))(x)
# x = Conv1D(32, 3, padding='causal')(middle_layer)
# x = Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(x)
# x = Conv1D(64, 3, padding='causal')(x)
# x = Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.5), merge_mode='concat')(x)
# output_layer = TimeDistributed(Dense(input_size, activation='sigmoid'))(x)
# model = Model(input_layer, output_layer)
# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])

In [ ]:
# input_size = 1 + notes + 8 + 16
# input_layer = Input((256, input_size))
# x = Bidirectional(LSTM(256, recurrent_dropout=0.5, return_sequences=True), merge_mode='concat')(input_layer)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = MaxPool1D(2)(x)
# # x = Conv1D(64, 3, padding='causal')(x)
# # x = Flatten()(x)
# x = Bidirectional(LSTM(256, recurrent_dropout=0.5), merge_mode='concat')(input_layer)
# output_layer = Dense(input_size, activation='sigmoid')(x)
# model = Model(input_layer, output_layer)
# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])

In [ ]:
# model.summary()

In [ ]:
# model.fit_generator(generator(train_data), 128, 128)

In [ ]:
# test = numpy.round(model.predict(X[numpy.newaxis, :, :])).astype(int)

In [ ]:
# test[:, :, 1:37]

In [ ]:
data = load_chords('/home/santiago/Projects/MusicGenerator/data/midi/')

In [ ]:
data

In [ ]:
input_size = 1 + notes + 8 + 16

In [ ]:
# input_layer = Input((None, input_size))
# x = Bidirectional(LSTM(128, return_sequences=True))(input_layer)
# x = Dropout(0.5)(x)
# x = TimeDistributed(Reshape((16, 16, 1)))(x)
# x = ConvLSTM2D(1, (3, 3), padding='same', return_sequences=True)(x)
# x = Activation('relu')(x)
# x = TimeDistributed(MaxPool2D(2))(x)
# x = ConvLSTM2D(1, (3, 3), return_sequences=False)(x)
# x = Activation('relu')(x)
# x = Flatten()(x)
# output_layer = Dense(input_size, activation='sigmoid')(x)
# model = Model(input_layer, output_layer)
# model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
# numpy.round(model.predict(train_data[0][0])).astype(int)

In [ ]:
# train_data[1][0]

In [ ]:
# input_layer2 = Input((None, input_size))
# x2 = Bidirectional(LSTM(128, return_sequences=True, weights=model.layers[1].get_weights()))(input_layer2)
# x2 = Dropout(0.5)(x2)
# x2 = TimeDistributed(Reshape((16, 16, 1)))(x2)
# x2 = ConvLSTM2D(1, (3, 3), padding='same', return_sequences=True, weights=model.layers[4].get_weights())(x2)
# x2 = Activation('relu')(x2)
# x2 = TimeDistributed(MaxPool2D(2))(x2)
# x2 = ConvLSTM2D(1, (3, 3), return_sequences=False, weights=model.layers[7].get_weights())(x2)
# output_layer2 = Activation('relu')(x2)
# model2 = Model(input_layer2, output_layer2)
# model2.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# model2.summary()

In [ ]:
# skimage.io.imshow(model2.predict(train_data[0][21]).reshape((3, 4, 3)))

In [3]:
input_layer = Input((None, 61))
x = Bidirectional(LSTM(128, return_sequences=True))(input_layer)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(128, return_sequences=False))(x)
x = Dropout(0.5)(x)
output_layer = Dense(input_size, activation='sigmoid')(x)
model = Model(input_layer, output_layer)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

AttributeError: can't set attribute

In [ ]:
model.fit_generator(generator(data), 128, 128)

In [ ]:
numpy.round(model.predict(data[0][0][2])).astype(int)

In [ ]:
input_layer2 = Input((None, input_size))
x2 = Bidirectional(LSTM(128, return_sequences=True, weights=model.layers[1].get_weights()))(input_layer2)
x2 = Dropout(0.5)(x2)
x2 = Bidirectional(LSTM(128, return_sequences=False, weights=model.layers[3].get_weights()))(x2)
output_layer2 = Dropout(0.5)(x2)
model2 = Model(input_layer2, output_layer2)
model2.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(100, 100))
skimage.io.imshow(model2.predict(data[0][0][6]))

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(model2.predict(data[1][0][100]), aspect='auto')

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(model2.predict(data[0][0][19]), aspect='auto')

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(model2.predict(data[0][0][20]), aspect='auto')

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(model2.predict(data[1][0][100]), aspect='auto')

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(model2.predict(data[1][0][1]), aspect='auto')

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(numpy.round(model.predict(data[0][0][0])), aspect='auto')

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(numpy.round(model.predict(data[0][0][0])), aspect='auto')

In [ ]:
test = numpy.round(model.predict(data[0][0][0]))

In [ ]:
test2 = numpy.round(model.predict(test[numpy.newaxis, :, :]))

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(test2, aspect='auto')

In [ ]:
test3 = numpy.round(model.predict(numpy.vstack([test, test2])[numpy.newaxis, :, :]))

In [ ]:
plt.figure(figsize=(20, 4))
skimage.io.imshow(test3, aspect='auto')

In [ ]:
##############################################################################################

In [ ]:
original_dim = 61
intermediate_dim = 128
latent_dim = 16 # must be even
epsilon_std = 1.0
timesteps = 32

In [ ]:
output_tensor = K.placeholder(shape=(None, 61))

In [ ]:
x = Input(shape=(timesteps, original_dim))
h = Bidirectional(LSTM(128, return_sequences=True))(x)
h = Dropout(0.5)(h)
z_mean = Bidirectional(LSTM(latent_dim // 2, return_sequences=False))(h)
z_log_var = Bidirectional(LSTM(latent_dim // 2, return_sequences=False))(h)
repeat_z_mean = RepeatVector(timesteps)(z_mean)
repeat_z_log_var = RepeatVector(timesteps)(z_log_var)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([repeat_z_mean, repeat_z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Bidirectional(LSTM(intermediate_dim, return_sequences=True))
decoder_mean = TimeDistributed(Dense(original_dim, activation='sigmoid'))
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)


# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean):
        xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        return x


y = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, y)
vae.compile(optimizer='adam', loss=None)